<a href="https://colab.research.google.com/github/rishabh-0042/American_Express_Hackathon_25-2025/blob/main/Blending_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

X_test = pd.read_parquet('/content/drive/MyDrive/AMEXChallenge/X_test.parquet')
X_train = pd.read_parquet('/content/drive/MyDrive/AMEXChallenge/X_train.parquet')
y_train = pd.read_parquet('/content/drive/MyDrive/AMEXChallenge/y_train.parquet')

In [3]:
!pip install lightgbm catboost xgboost scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 3.3 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import pandas as pd

# Convert y_train to numeric type
y_train_numeric = y_train.values.ravel().astype(int)

# Train LightGBM
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train_numeric)
lgbm_pred_proba = lgbm.predict_proba(X_test)[:, 1]

# Train CatBoost
catboost = cb.CatBoostClassifier(verbose=0)
catboost.fit(X_train, y_train_numeric)
catboost_pred_proba = catboost.predict_proba(X_test)[:, 1]

# Train RandomForest
rf = RandomForestClassifier()
rf.fit(X_train, y_train_numeric)
rf_pred_proba = rf.predict_proba(X_test)[:, 1]

# Train LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train_numeric)
lr_pred_proba = lr.predict_proba(X_test)[:, 1]

# Train XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train_numeric)
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Store predictions in a DataFrame
predictions = pd.DataFrame({
    'LightGBM': lgbm_pred_proba,
    'CatBoost': catboost_pred_proba,
    'RandomForest': rf_pred_proba,
    'LogisticRegression': lr_pred_proba,
    'XGBoost': xgb_pred_proba
})

display(predictions.head())

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from scipy.stats import uniform, randint

# 1. Imbalance Handling (SMOTE)
print("Applying SMOTE...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train.values.ravel())
print("SMOTE applied.")

# 2. Model Definitions and Parameter Ranges
print("Defining models and parameter ranges...")
# LightGBM
lgbm = lgb.LGBMClassifier(random_state=42)
lgbm_param_dist = {
    'num_leaves': randint(20, 150),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.15),
    'n_estimators': randint(50, 500),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'reg_alpha': uniform(0, 0.1),
    'reg_lambda': uniform(0, 0.1),
}

# CatBoost
catboost = cb.CatBoostClassifier(random_state=42, verbose=0)
catboost_param_dist = {
    'iterations': randint(50, 500),
    'learning_rate': uniform(0.01, 0.15),
    'depth': randint(3, 15),
    'l2_leaf_reg': uniform(1, 10),
    'border_count': randint(32, 255),
}

# XGBoost
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_param_dist = {
    'n_estimators': randint(50, 500),
    'learning_rate': uniform(0.01, 0.15),
    'max_depth': randint(3, 15),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 0.5),
    'reg_alpha': uniform(0, 0.1),
    'reg_lambda': uniform(0, 0.1),
}

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf_param_dist = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(3, 15),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'bootstrap': [True, False],
}

models = {
    'LightGBM': (lgbm, lgbm_param_dist),
    'CatBoost': (catboost, catboost_param_dist),
    'XGBoost': (xgb_model, xgb_param_dist),
    'RandomForest': (rf, rf_param_dist),
    # Logistic Regression is typically not tuned with RandomizedSearchCV on this scale
}
print("Models and parameter ranges defined.")

# 3. Quick Tuning (with speed-up tips)
print("Starting quick tuning...")
tuned_estimators = {}
n_iter = 10 # Reduced iterations for tuning
cv_folds = 2 # Reduced folds for tuning

# Subsample for tuning
subsample_ratio = 0.1 # Reduced subsample fraction
subsample_size = int(len(X_train_resampled) * subsample_ratio)
subsample_indices = np.random.choice(len(X_train_resampled), size=subsample_size, replace=False)
X_train_subsampled = X_train_resampled[subsample_indices]
y_train_subsampled = y_train_resampled[subsample_indices]

for name, (model, param_dist) in models.items():
    print(f"Tuning {name}...")
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                       n_iter=n_iter, cv=StratifiedKFold(n_splits=cv_folds),
                                       scoring='roc_auc', random_state=42, n_jobs=-1)
    random_search.fit(X_train_subsampled, y_train_subsampled)
    tuned_estimators[name] = random_search.best_estimator_
    print(f"Tuning complete for {name}. Best params: {random_search.best_params_}")

print("Quick tuning complete.")

# 4. Full Retrain
print("Starting full retraining on SMOTE-augmented data...")
retrained_models = {}
for name, model in tuned_estimators.items():
    print(f"Retraining {name}...")
    retrained_models[name] = model.fit(X_train_resampled, y_train_resampled)
    print(f"Retraining complete for {name}.")
print("Full retraining complete.")

# 5. Blending
print("Computing predictions and blending...")
predictions = pd.DataFrame()
for name, model in retrained_models.items():
    predictions[name] = model.predict_proba(X_test)[:, 1]

# Simple average blending
final_predictions = predictions.mean(axis=1)
print("Predictions computed and blended.")

# Display final predictions (or a sample)
print("\nSample of final blended predictions:")
display(final_predictions.head())

Applying SMOTE...


In [9]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from scipy.stats import uniform, randint

# ============================
# 1. Load and Balance
# ============================


smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# ============================
# 2. Define Models & Distributions
# ============================
models = {
    'LightGBM': (
        LGBMClassifier(
            n_jobs=-1, random_state=42,
            learning_rate=0.05, num_leaves=64,
            n_estimators=200, subsample=0.8,
            colsample_bytree=0.8
        ), {
            'num_leaves': randint(31, 128),
            'learning_rate': uniform(0.01, 0.1),
            'n_estimators': randint(50, 201)
        }
    ),
    'CatBoost': (
        CatBoostClassifier(
            task_type='CPU',
            random_state=42, verbose=0,
            learning_rate=0.05, depth=6, iterations=200
        ), {
            'learning_rate': uniform(0.01, 0.1),
            'depth': randint(4, 10),
            'iterations': randint(50, 301)
        }
    ),
    'XGBoost': (
        XGBClassifier(
            random_state=42, use_label_encoder=False,
            eval_metric='logloss', n_jobs=-1,
            learning_rate=0.05, max_depth=5, n_estimators=200
        ), {
            'learning_rate': uniform(0.01, 0.1),
            'max_depth': randint(3, 8),
            'n_estimators': randint(50, 201)
        }
    ),
    'RandomForest': (
        RandomForestClassifier(
            n_jobs=-1, random_state=42,
            n_estimators=200, max_depth=10,
            min_samples_split=5, min_samples_leaf=3
        ), {
            'n_estimators': randint(50, 201),
            'max_depth': randint(5, 15),
            'min_samples_split': randint(2, 11),
            'min_samples_leaf': randint(1, 6)
        }
    )
}

In [5]:
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 5.5 MB/s eta 0:00:00


In [11]:
# ============================
# 3. Quick Hyperparameter Tuning
# ============================
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
n_iter = 5
sub_frac = 0.1
idx = np.random.choice(len(X_res), size=int(len(X_res)*sub_frac), replace=False)
X_sub, y_sub = X_res.iloc[idx], y_res.iloc[idx].astype(int) # Convert y_sub to int

best_models = {}
for name, (estimator, dist) in models.items():
    print(f"Tuning {name}...")
    rs = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=dist,
        n_iter=n_iter,
        cv=cv,
        scoring='roc_auc',
        random_state=42,
        n_jobs=-1
    )
    rs.fit(X_sub, y_sub)
    best_models[name] = rs.best_estimator_
    print(f"-> {name} best AUC: {rs.best_score_:.4f}\n")

# ============================
# 4. Retrain on Full Data
# ============================
for name, model in best_models.items():
    print(f"Retraining {name} on full dataset...")
    model.fit(X_res, y_res.astype(int))
# ============================
# 5. Prediction & Blending
# ============================
preds = pd.DataFrame({
    name: model.predict_proba(X_test)[:, 1]
    for name, model in best_models.items()
})
weights = {
    'LightGBM': 0.35,
    'CatBoost': 0.35,
    'XGBoost': 0.20,
    'RandomForest': 0.10
}
final_pred = sum(preds[name] * w for name, w in weights.items())

Tuning LightGBM...
[LightGBM] [Info] Number of positive: 73112, number of negative: 73510
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.272190 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 68603
[LightGBM] [Info] Number of data points in the train set: 146622, number of used features: 338
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498643 -> initscore=-0.005429
[LightGBM] [Info] Start training from score -0.005429
-> LightGBM best AUC: 0.9964

Tuning CatBoost...
-> CatBoost best AUC: 0.9946

Tuning XGBoost...
-> XGBoost best AUC: 0.9963

Tuning RandomForest...
-> RandomForest best AUC: 0.9918

Retraining LightGBM on full dataset...
[LightGBM] [Info] Number of positive: 733113, number of negative: 733113
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.824263 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bin

In [12]:
aucs = {
    'LightGBM': 0.9964,
    'CatBoost': 0.9946,
    'XGBoost': 0.9963,
    'RandomForest': 0.9918
}

scores = {m: auc - 0.5 for m, auc in aucs.items()}
total  = sum(scores.values())
weights = {m: s/total for m, s in scores.items()}

print("Blend weights based on AUC:", weights)

final_pred = sum(preds[name] * w for name, w in weights.items())

display(final_pred.head())

Blend weights based on AUC: {'LightGBM': 0.2508210802890203, 'CatBoost': 0.24991157596887478, 'XGBoost': 0.2507705522712344, 'RandomForest': 0.24849679147087061}


,0
0,0.093988
1,0.130298
2,0.822124
3,0.090113
4,0.082703


In [13]:
import pandas as pd

submission_df = pd.read_csv('/content/drive/MyDrive/AMEXChallenge/r2_submission_file<22je0664>.csv')

submission_df_copy = submission_df.copy()

submission_df_copy['pred'] = final_pred.values

display(submission_df_copy.head())

,id1,id2,id3,id5,pred
0,1362907_91950_16-23_2023-11-04 18:56:26.000794,1362907,91950,2023-11-04,0.093988
1,1082599_88356_16-23_2023-11-04 06:08:53.373,1082599,88356,2023-11-04,0.130298
2,1888466_958700_16-23_2023-11-05 10:07:28.000725,1888466,958700,2023-11-05,0.822124
3,1888971_795739_16-23_2023-11-04 12:25:28.244,1888971,795739,2023-11-04,0.090113
4,1256369_82296_16-23_2023-11-05 06:45:26.657,1256369,82296,2023-11-05,0.082703


In [14]:

submission_df_copy.to_csv('/content/drive/MyDrive/AMEXChallenge/r2_submission_file<22je0664>2.csv', index=False)

print("Modified submission file saved successfully.")

Modified submission file saved successfully.
